## Springer API Simple Request Example

Example of a simple request searching for the combination Machine Learning AND (boolean) coronary in the configured metadata fields and abstracts of the articles in the database.

In [2]:
import requests

adress = 'http://api.springer.com/meta/v1/json?'
queryterms = 'q=Coronary AND Machine Learning AND year:2015'
api_key = '&api_key=f1333dfd6f52767d0a77099010fbefbc'
max_record = '&p=100'
start_record='&s=1'
year = ' year:2017'
records_type = ' type:Journal'

query = queryterms
url = adress + query + api_key + start_record + max_record

r = requests.get(url)

print("request: \n", r.url, "\n")
print("results: ", r.json()['result'])
print("keys: \n", r.json().keys())
print("\nArticle Fields:\n ", r.json().get('records')[0].keys())


request: 
 http://api.springer.com/meta/v1/json?q=Coronary%20AND%20Machine%20Learning%20AND%20year:2015&api_key=f1333dfd6f52767d0a77099010fbefbc&s=1&p=100 

results:  [{'total': '371', 'start': '1', 'pageLength': '100', 'recordsDisplayed': '100'}]
keys: 
 dict_keys(['query', 'apiKey', 'result', 'records', 'facets'])

Article Fields:
  dict_keys(['identifier', 'url', 'title', 'creators', 'publicationName', 'issn', 'eissn', 'openaccess', 'journalid', 'doi', 'publisher', 'publicationDate', 'onlineDate', 'coverDate', 'printDate', 'volume', 'number', 'issuetype', 'topicalCollection', 'startingPage', 'endingPage', 'copyright', 'genre', 'abstract'])


In [3]:
import re
import requests
from datetime import datetime


class Springer_Searcher():
    
    api_key = 'f1333dfd6f52767d0a77099010fbefbc'
    address = 'http://api.springer.com/meta/v1/json?'
    
    def __init__(self, queryterms: list=None, api_key: str=None, address: str=None): 
        self.articles_found = []
        if queryterms: 
            self.queryterms = queryterms
        if api_key: 
            self.api_key = api_key
        if address: 
            self.address = address
            
    def search(self, queryterms: list=None,max_records: int=100,start_record: int=1, year: int=None):
        
        """
        @param queryterms: list of lists. Terms within the same list are
            separated by an OR. Lists are separated by an AND
        @param max_records: Number of results to return in this request.
        @param start_record: Return results starting at the number specified.
        @param year: limit to articles/chapters published from a particular year to actual year. 
                     If left blank will serach all year
        """
        
        if not queryterms: 
            queryterms = self.queryterms
        
        
        
        formated_query = " AND ".join(["(%s)" % " OR ".join(term)for term in queryterms])
        
        url = self.address + "q=" + formated_query 
        
        if year:
            url += ' year:' + str(year)
            if max_records: 
                url += '&p=' + str(max_records)
            if start_record: 
                url += '&s=' + str(start_record)


            url += '&api_key=' + self.api_key

            r = requests.get(url)

            lst = re.findall("'\S+'", str(r.json().get('result')))
            index_of_total = lst.index("'total'")


            while(year<=datetime.today().year):
                r = requests.get(url)

                lst = re.findall("'\S+'", str(r.json().get('result')))
                index_of_total = lst.index("'total'")
                total_records = int (lst[index_of_total+1][1:-1])

                print("Requisition: ")
                print("\n", r.url, "\n")
                print("Articles found in ",year,": ",total_records)
                print(round((total_records/max_records)+0.4999), " requests needed...")
                self.articles_found += r.json().get('records')
                print(year, " request completed...")
                while ((start_record + max_records ) < total_records):
                    url = url.replace('&s='+str(start_record), 
                                      '&s='+str(start_record+max_records) )
                    r = requests.get(url)
                    self.articles_found += r.json().get('records')
                    start_record += max_records
                url = url.replace('year:'+str(year),'year:'+str(year+1))
                year += 1
            
            return self.articles_found
        
        if max_records: 
            url += '&p=' + str(max_records)
        if start_record: 
            url += '&s=' + str(start_record)
        
        
        url += '&api_key=' + self.api_key
        
        r = requests.get(url)
        
        lst = re.findall("'\S+'", str(r.json().get('result')))
        index_of_total = lst.index("'total'")
        total_records = int (lst[index_of_total+1][1:-1])
        
        print("Requisição: ")
        print("\n", r.url, "\n")
        print(round((total_records/max_records)+0.4999), " requests needed...")
        self.articles_found = r.json().get('records')
        print("a request completed...")
        while (start_record + max_records) < total_records:
            url = url.replace('&s='+str(start_record), 
                              '&s='+str(start_record+max_records) )
            r = requests.get(url)
            self.articles_found += r.json().get('records')
            start_record += max_records
            print("a request completed...")
        
        return self.articles_found

## Using the Springer_Searcher class

In [4]:
technology_queryterms = [
    'machine learning', 'deep learning', 'artificial intelligence', 
    'neural network', 'scoring system'
]

health_queryterms = [
    'coronary artery disease', 'chest pain', 'heart disease', 'MACE', 
    'Acute Cardiac Complications'
]

queryterms = [technology_queryterms, health_queryterms]
year = 2015

springer_searcher = Springer_Searcher()
articles = springer_searcher.search(queryterms=queryterms,year=year)

print("\nTotal found:", len(articles))
print("\nArticle fields\n", articles[0].keys())
print("\nArticle List\n")
for i, article in enumerate(articles): 
    print(i+1, " - ", article.get('title'))

Requisition: 

 http://api.springer.com/meta/v1/json?q=(machine%20learning%20OR%20deep%20learning%20OR%20artificial%20intelligence%20OR%20neural%20network%20OR%20scoring%20system)%20AND%20(coronary%20artery%20disease%20OR%20chest%20pain%20OR%20heart%20disease%20OR%20MACE%20OR%20Acute%20Cardiac%20Complications)%20year:2015&p=100&s=1&api_key=f1333dfd6f52767d0a77099010fbefbc 

Articles found in  2015 :  27
1  requests needed...
2015  request completed...
Requisition: 

 http://api.springer.com/meta/v1/json?q=(machine%20learning%20OR%20deep%20learning%20OR%20artificial%20intelligence%20OR%20neural%20network%20OR%20scoring%20system)%20AND%20(coronary%20artery%20disease%20OR%20chest%20pain%20OR%20heart%20disease%20OR%20MACE%20OR%20Acute%20Cardiac%20Complications)%20year:2016&p=100&s=1&api_key=f1333dfd6f52767d0a77099010fbefbc 

Articles found in  2016 :  28
1  requests needed...
2016  request completed...
Requisition: 

 http://api.springer.com/meta/v1/json?q=(machine%20learning%20OR%20deep%2

In [14]:
from functools import reduce

item = articles[2]

all_item = list()

for item in articles:
    dados = dict()

    l = lambda x, y: f'{x}; {y["creator"]}'
    lista = item.get('creators')

    url  = [x['value'] for x in item.get('url') if x['format'] == 'html'][0]
    
    try:
        dados['autores'] = reduce(l, lista[0:], lista[0]['creator'])
    except:
        dados['autores'] = None
    dados['titulo'] = item['title']
    dados['doi'] = item['doi']
    dados['revista'] = item['publicationName']
    dados['html_url'] = url
    dados['data'] = datetime.strptime(item['publicationDate'], '%Y-%m-%d')
    dados['resumo'] = item['abstract']
    
    all_item.append(dados)
all_item

[{'autores': 'Li, Xi-ming; Li, Xi-ming; Li, Feng; Liu, Zhong-kai; Shao, Ming-tao',
  'data': datetime.datetime(2015, 12, 1, 0, 0),
  'doi': '10.1631/jzus.B1500030',
  'html_url': 'http://link.springer.com/openurl/fulltext?id=doi:10.1631/jzus.B1500030',
  'resumo': 'AbstractObjectiveTo explore the relationship of postoperative cognitive dysfunction (POCD) in one-lung ventilation (OLV) patients and regional cerebral oxygen saturation (rSO_2).MethodsTwenty-nine male and twenty-one female cases of OLV received thoracic surgery, with American Standards Association (ASA) physical status being at Grades I–III. Neuropsychological tests were performed on the day before operation and 7 d after operation, and there was an intraoperative continuous monitoring of rSO_2. The values of rSO_2 before anesthesia induction ( t _1), at the beginning of OLV ( t _2), and at the time of OLV 30 min ( t _3), OLV 60 min ( t _4), the end of OLV ( t _5), and the end of surgery ( t _6) were recorded. The intraoper

In [6]:
lista

[{'creator': 'Huang, Zhengxing'},
 {'creator': 'Dong, Wei'},
 {'creator': 'Ji, Lei'},
 {'creator': 'Duan, Huilong'}]

In [ ]:
reduce(funcao, lista, primeiro)